In [1]:
import csv
import copy
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
from feature import Feature
import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatool.preprocess import  Preprocessor
from datatool.maneger import DataManager

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
path = "../corpus/data_augment.csv"

In [4]:
data = []
with open(path) as f:
    reader = csv.reader(f)
    for row in reader:
        data.append(row)

In [5]:
def make_classes_dict(data):
    classes_dict = dict()
    for row in data:
        c = row[1]
        if c not in classes_dict:
            classes_dict[c] = len(classes_dict)
    return classes_dict

In [6]:
def class_filter(data, remain_classes:list):
    new_data = []
    for row in data:
        c = row[1]
        if c in remain_classes:
            new_data.append(row)
    return new_data

In [7]:
classes_dict_ = make_classes_dict(data)

In [8]:
remain_classes = "how what when where who why YN plain".split()
data_n = class_filter(data, remain_classes)
classes_dict = dict(zip(remain_classes, list(range(len(remain_classes)))))

In [9]:
def extract_X_y(data, classes_dict):
    X = []
    y = []
    for d in data:
        X.append(d[0])
        y.append(classes_dict[d[1]])
    return X, y

In [10]:
import random
def extract_X_y_limit(data, classes_dict:dict, limit=10):
    data_ = random.sample(data, len(data))
    X = []
    y = []
    each_len = dict(zip(classes_dict.keys(), [0]*len(classes_dict)))
    for d in data_:
        if each_len[d[1]] <= limit:
            X.append(d[0])
            y.append(classes_dict[d[1]])
            each_len[d[1]] += 1
    return X, y

In [11]:
# X, y = extract_X_y(data_n, classes_dict)

In [12]:
X, y = extract_X_y_limit(data_n, classes_dict)

In [13]:
len(X)

88

In [14]:
X_train_str, X_test_str, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)

In [15]:
F = Feature()
F.set_preprocessor(Preprocessor())
F.make_features(X_train_str)

X_train = []
X_test = []
for i, x_t_str in enumerate( X_train_str ):
    x = F.featurization(x_t_str)
    X_train.append(x)
for i, x_t_str in enumerate( X_test_str ):
    x = F.featurization(x_t_str)
    X_test.append(x)
X_train = np.array(X_train)
X_test = np.array(X_test)

300


In [16]:
F_path = "../X_y_data/"
F_name = "typeClassify_F2.pickle"
import dill
import pickle
# featureM = DataManager(F_path, format_="dill")
with open(F_path+F_name, "wb") as f:
    pickle.dump(F, f)

In [17]:
lr = LogisticRegression(solver='sag', max_iter=100)
lr.fit(X_train, y_train)

/home/yamada/.local/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(solver='sag')

In [18]:
y_pred = lr.predict(X_test)

In [19]:
print(classes_dict)
dis = 30
for y_p, x_s in zip(y_pred[:dis], X_test_str[:dis]):
    print("{0} : {1}".format(y_p, x_s))

{'how': 0, 'what': 1, 'when': 2, 'where': 3, 'who': 4, 'why': 5, 'YN': 6, 'plain': 7}
6 : パイを3つもらおう。
4 : 取り残されることの恐怖から、つまりFOMOから、通知やメッセージに絶えず反応する循環に陥っているのです。
0 : どうしてわかったの？
2 : あなたの名前は何ですか？
2 : どんな所がお好みですか？
3 : きみがぼくのために作ってくれたの？
0 : ハワードさん、タクシー呼びましょうか？
0 : あなたたち、なぜ笑っているの？
2 : いちばん好きなクリスマス・ソングは何ですか？
3 : いつお店に行ったの？
0 : あなたたちは、どうしてスナップドラゴンの手下になっているの？
4 : どういうこと？
3 : その店はどこにあるの？
4 : ウィンター山脈では一年中雪が降る。
3 : ホームセンターはどこなの？
4 : 確認願えますか？
1 : これは正確には何でしたっけ？
2 : いつ建てられたのですか？
4 : あの女の子はだれ？
1 : 何歳なの？
5 : わたしたち、ここで1年くらい前に出会ったんだよね、オリバー。
4 : どこでフェイスペインティングを習ったの、レイチェル?
3 : 誰がそのおいしそうなチョコレートケーキを作ったの？
2 : 和太鼓の稽古はないの？
5 : マリーゴールド畑に入ると、そう簡単には出られないのです。
0 : 俳優たちの顔の表情を見た？
1 : 音楽が好きなの？


In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[2 1 0 0 0 0 0 0]
 [0 1 3 0 1 0 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 0 2 1 0 0 0]
 [0 0 0 1 1 0 0 0]
 [2 0 0 0 0 0 0 0]
 [1 1 1 1 1 0 0 0]
 [0 0 0 0 2 2 1 0]]
accuracy =  0.25925925925925924


In [21]:
model_path = "../models/"
model_name = "typeClassify_M2.pickle"
modelM = DataManager(model_path)
print(model_name)

typeClassify_M2.pickle


In [22]:
modelM.save_data(model_name, lr)

success save : ../models/typeClassify_M2.pickle
